## Test local filss of train.py

In [34]:
# ! python train.py --run_mode_container True
# ! python trainSI.py --run_mode_container True
# ! python train.py 



<h2>Prepare for a train file

In [35]:
# ! ls ./train.py -al
# ! cp ./train.py train_container/train
# ! chmod +x train_container/train
# ! chmod +x train_container/build_and_push.sh
# ! ls train_container/train -al

! cp ./DDDQNNet.py train_container/
! cp ./utils.py train_container/
! cp ./trainDDDQN.py train_container/train

! chmod +x train_container/train
! ls train_container -al

total 96
drwxrwxr-x  4 ec2-user ec2-user  4096 Feb 23 09:16 .
drwxrwxr-x 11 ec2-user ec2-user  4096 Feb 23 10:56 ..
-rw-rw-r--  1 ec2-user ec2-user 10510 Feb 23 10:56 DDDQNNet.py
-rw-rw-r--  1 ec2-user ec2-user  1014 Feb 23 09:10 Dockerfile
-rw-rw-r--  1 ec2-user ec2-user  1258 Feb 22 08:22 Dockerfile.cpu
-rw-rw-r--  1 ec2-user ec2-user   175 Feb 23 09:16 hyperparameters.json
drwxrwxr-x  2 ec2-user ec2-user  4096 Feb 23 09:16 .ipynb_checkpoints
drwxrwxr-x  2 ec2-user ec2-user  4096 Feb 22 08:22 SI_ROMS
-rwxrwxr-x  1 ec2-user ec2-user 17342 Feb 23 10:56 train
-rw-rw-r--  1 ec2-user ec2-user 17317 Feb 23 07:59 trainDDDQN.py
-rw-rw-r--  1 ec2-user ec2-user 13497 Feb 23 10:56 utils.py


<h2>Test local container 

In [36]:
import os
os.environ['train_container_name']= "train-space-invader-dddqn-gpu"
train_container_name = "train-space-invader-dddqn-gpu"

In [37]:
%%sh

# The name of our algorithm
algorithm_name=$train_container_name

cd train_container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

echo $region

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 763104351884 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

# docker push ${fullname}

us-east-2
Login Succeeded
Login Succeeded
Sending build context to Docker daemon    107kB
Step 1/14 : FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/tensorflow-training:1.15.0-gpu-py36-cu100-ubuntu18.04
 ---> 86d19102e250
Step 2/14 : RUN pip install gym-retro
 ---> Using cache
 ---> 70d9ddf74b9c
Step 3/14 : RUN pip install -U scikit-image
 ---> Using cache
 ---> 7e5ed48b3743
Step 4/14 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 68f73630e724
Step 5/14 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 79060c190c29
Step 6/14 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 13f11b49a587
Step 7/14 : COPY hyperparameters.json /opt/ml/input/config/
 ---> Using cache
 ---> 90b534ef63fb
Step 8/14 : COPY SI_ROMS/ /opt/ml/program
 ---> Using cache
 ---> 916f5dfb4df3
Step 9/14 : COPY train /opt/ml/code/
 ---> 27a89819f5b5
Step 10/14 : COPY DDDQNNet.py /opt/ml/code/
 ---> bcba068a2a13
Step 11/14 : COPY utils.py /opt/ml/code/
 ---> fd287d7080dc
Step 12/14 : WORKDIR /opt/

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [38]:
# # !docker run $train_container_name python train
!docker run $train_container_name train  --run_mode_container=True

run_mode_container:  True
The mode is NO container
model_path:  /opt/ml/model/model.ckpt
The size of our frame is:  Box(210, 160, 3)
The action size is:  8

2020-02-23 10:56:32.468929: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/openmpi/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-02-23 10:56:32.468959: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2020-02-23 10:56:32.468976: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5d4145c986a2): /proc/driver/nvidia/version does not exist
2020-02-23 10:56:32.469194: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-02-23 10:56:32.495911: I tenso

<h2> Push to ECR

In [39]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
import sagemaker as sage

role = get_execution_role()
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, train_container_name)
print("docker image: ", image)
# 057716757052.dkr.ecr.us-east-2.amazonaws.com/train-space-invader-gpu-emoji

# ! echo {image}
! docker push {image}

# %%sh
# cd train_container
# ./build_and_push.sh $train_container_name
# # # 057716757052.dkr.ecr.us-east-2.amazonaws.com/191204-text:latest


docker image:  057716757052.dkr.ecr.us-east-2.amazonaws.com/train-space-invader-dddqn-gpu:latest
The push refers to repository [057716757052.dkr.ecr.us-east-2.amazonaws.com/train-space-invader-dddqn-gpu]

de9056d9: Preparing 
387728e2: Preparing 
14d103c3: Preparing 
80c6d201: Preparing 
1f6064b7: Preparing 
7c3b0f10: Preparing 
fd3b3f9d: Preparing 
0397b295: Preparing 
363c7bbd: Preparing 
a1765c0f: Preparing 
db1e5fa3: Preparing 
376d2ac8: Preparing 
dc4932e7: Preparing 
f0752ab0: Preparing 
1181bd36: Preparing 
772b5b08: Preparing 
3c24a493: Preparing 
b226ebe0: Preparing 
3e4fc209: Preparing 
c3b0f10: Waiting g 
71fb55e8: Preparing 
4c09fe29: Preparing 
d3b3f9d: Waiting g 
8c2b9bbc: Preparing 
c4932e7: Waiting g 
397b295: Waiting g 
1c774a30: Preparing 
a4b22186: Preparing 
0752ab0: Waiting g 
b5a53aac: Preparing 
c8e5063e: Preparing 
7c529ced: Layer already exists 1kBlatest: digest: sha256:259d15761c66deb65cbeb6496cf14913f989eb21e666087b62cc87a5da9cced9 size: 7008
